# Data Management and Boilerplate Code

## Data Separation

The data is separated into training, validation, and testing pools using a random number generator always seeded with the same value (so separation occurs the same way every time)

In [1]:
import pandas as pd
df = pd.read_csv("../data/preprocessed.csv")
df.drop(["Unnamed: 0"], axis=1, inplace=True)

# Pull out validation (15%) and test (15%) data from training (70%) data
df_valid_test = df.sample(frac=0.3, random_state=0xda)
df_training = df.drop(df_valid_test.index)

df_valid = df_valid_test.sample(frac=0.5, random_state=0xdb)
df_test = df_valid_test.drop(df_valid.index, axis=0)

df_training.head()

,age,discharged_home,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,...,insulin_No,insulin_Steady,insulin_Up,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,OUTPUT_<30,OUTPUT_>30,OUTPUT_NO
1,0.15,1,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.000000,0.533333,...,0,0,1,1,0,0,1,0,1,0
2,0.25,1,0.076923,0.076336,0.833333,0.1500,0.047619,0.0,0.047619,0.333333,...,1,0,0,0,1,0,1,0,0,1
3,0.35,1,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.000000,0.400000,...,0,0,1,1,0,0,1,0,0,1
4,0.45,1,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.000000,0.266667,...,0,1,0,1,0,0,1,0,0,1
7,0.75,1,0.307692,0.549618,0.000000,0.1375,0.000000,0.0,0.000000,0.466667,...,1,0,0,0,1,0,1,0,1,0


## Batches and Categorization

The following class can be used as an iterable in a `for` loop. It takes a dynamic collection of data separated into pools (e.g. categories like readmission vs. no readmission) and produces a batch which samples equally frequently from each pool. The iterable stops when all data in the largest pool have been iterated over once

In [2]:
N_INPUTS = 67

class TrainingBatches:

    def __init__(self, nn, batch_size=16):
        self.nn = nn
        self.batch_size = batch_size
        self.bounds = max([d.shape[0] for d in nn.training_data])
        self.it = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.it >= self.bounds:
            raise StopIteration

        output = np.concatenate([self.getSlice(d, self.it, self.batch_size) for d in self.nn.training_data])
        self.it += self.batch_size
        return output[:, :N_INPUTS], output[:, N_INPUTS:]

    # Code must work with Python 2 and 3
    next = __next__

    def getSlice(self, array, start, length):
        start = start % array.shape[0]
        if start + length >= array.shape[0]:
            return np.concatenate((array[start:], array[:(start + length) % array.shape[0]]), axis=0)
        else:
            return array[start:start + length]

In [3]:
class CategoricalPreprocessor:
    
    def __init__(self, df, output_categories):
        self.output_categories = output_categories
        self.df = df
        self.is_modified = False
        
    def modifyDatafile(self):
        
        if self.is_modified:
            return self.df
        
        self.is_modified = True
        
        if self.output_categories == "any":
            self.df["OUTPUT_ANY"] = self.df["OUTPUT_<30"] + self.df["OUTPUT_>30"]
            self.df.drop(["OUTPUT_<30", "OUTPUT_>30"], axis=1, inplace=True)

        elif self.output_categories == "rapid":
            self.df["OUTPUT_NO"] = self.df["OUTPUT_>30"] + self.df["OUTPUT_NO"]
            self.df.drop(["OUTPUT_>30"], axis=1, inplace=True)
            
        return self.df
    
    def getArraysByOutput(self):
        
        if not self.is_modified:
            self.modifyDatafile()
            
        if self.output_categories == "three":
            return (
                self.df[self.df["OUTPUT_<30"] == 1].to_numpy().astype("float32"),
                self.df[self.df["OUTPUT_>30"] == 1].to_numpy().astype("float32"),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy().astype("float32")
            )

        elif self.output_categories == "any":
            return (
                self.df[self.df["OUTPUT_ANY"] == 1].to_numpy().astype("float32"),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy().astype("float32")
            )

        else:
            return (
                self.df[self.df["OUTPUT_<30"] == 1].to_numpy().astype("float32"),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy().astype("float32")
            )

# Neural Network

From study of existing research and experimentation with tuning hyperparameters, the following configuration gives pretty good results for this particular problem:

 * 70 neurons in 1st hidden layer, 20 neurons in 2nd hidden layer; both layers have sigmoidal activation functions to make transition to a zero-or-one function more natural
 * Grouping outputs into two categories: 1) the patient is readmitted within 30 days, and 2) the patient is not readmitted within 30 days; this is described in the code as "rapid"
 * Dropout rate of 0.1
 * Batch size of 128, 64 of each output category to have balanced sampling; this is implemented using the `TrainingBatches` class
 
In order to make the neural network more sparse (i.e. there are many weight parameters that are 0), the algorithm described by [Srivinas, et al.](https://arxiv.org/pdf/1611.06694.pdf) will be used.

A layer of weights $W_i$ is masked by a matrix of parameters $g_i^S \in \{0, 1\}^{n_i}$. $g_i^S$ is sampled as an interpretation of the most likely outcome of a bernoulli trial: $g_i > 0.5$. The optimization algorithm chooses evaluative parameters $\hat \theta = \{W, B\}$ and sparsification parameters $\hat \phi = \{g^S\}$ by minimizing the following function:

$$\mathrm{loss}(\hat y(\theta, \phi), y) + \lambda_1 \sum_{i=1}^m g_i(1 - g_i) + \lambda_2 \sum_{i=1}^m g_i$$

The sparsification parameters are represented as $g^S = \mathrm{bernoulli}(g)$ such that $g = \mathrm{clip}(x)$ and $\frac{dg^S}{dg} = 1$.

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Support for custom gradients

 * [StackOverflow](https://stackoverflow.com/questions/39921607/how-to-make-a-custom-activation-function-with-only-python-in-tensorflow)
 * [GitHub Gist](https://gist.github.com/harpone/3453185b41d8d985356cbe5e57d67342)

In [5]:
@tf.custom_gradient
def clip(x):
    def grad(dy):
        return 1
    return tf.clip_by_value(x, 0, 1), grad

@tf.custom_gradient
def bernoulli(x):
    def grad(dy):
        return 1
    return tf.dtypes.cast(tf.greater(x, 0.5), "float"), grad

## Sparse Tensors

The following function creates a random tensor with a certain sparsity. The parameters `zero` and `one` define the values for sparse and non-sparse elements.

Another function calculates the sparsity of a tensor.

In [6]:
SPARSITY_THRESHOLD = 0.5

def randomSparseMatrix(shape, sparsity, zero, one):
    return one - (one - zero) * np.random.binomial(1, sparsity, shape)

def sparsity(tensor):
    return sum(tf.less(tensor, SPARSITY_THRESHOLD)) / tensor.size

## Neural Network Generator

In [12]:
class NeuralNetwork:

    def __init__(self, df, df_validation, layer_sizes, output_categories, dropout, batch_size, initial_sparsity):
        self.dropout = dropout
        self.output_categories = output_categories
        self.batch_size = batch_size
        self.initial_sparsity = initial_sparsity
        
        # Prepare training data
        training_processor = CategoricalPreprocessor(df, output_categories)
        self.training_data = training_processor.getArraysByOutput()
        
        # Prepare training batches
        training_in_out = list(iter(TrainingBatches(self, 32)))
        self.training_in = np.concatenate([i for i, o in training_in_out])
        self.training_out = np.concatenate([o for i, o in training_in_out])
        
        # Prepare validation data
        validation_processor = CategoricalPreprocessor(df_validation, output_categories)
        self.validation_data = validation_processor.modifyDatafile().to_numpy()
        self.validation_in = self.validation_data[:, :N_INPUTS]
        self.validation_out = self.validation_data[:, N_INPUTS:]
        
        # Structure
        self.layer_sizes = (N_INPUTS,) + layer_sizes + (self.validation_out.shape[1],)
        
        # Variables
        self.X = tf.placeholder("float32", [None, self.layer_sizes[0]])
        self.Y = tf.placeholder("float32", [None, self.layer_sizes[-1]])
        self.W = [tf.Variable(
                tf.random_normal([self.layer_sizes[i], self.layer_sizes[i+1]]),
            dtype="float32") for i in range(len(self.layer_sizes) - 1)]
        self.B = [tf.Variable(
                tf.random_normal([sz]),
            dtype="float32") for sz in self.layer_sizes[1:]]
        self.Gx = [tf.Variable(
                randomSparseMatrix([self.layer_sizes[i], self.layer_sizes[i+1]], self.initial_sparsity, 0.49, 1.0),
            dtype="float32") for i in range(len(self.layer_sizes) - 1)]
        self.a = tf.constant(1.0, dtype="float32")
        self.L1 = tf.constant(0.01, dtype="float32")
        self.L2 = tf.constant(0.01, dtype="float32")
        
        # Make the classifier graph
        self.classifier = self.getClassifier()
        
        # Optimization
        self.loss = self.getLoss()
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(self.loss)    

        
    def getLoss(self):
        loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(self.Y, self.classifier))
        G = [clip(Gx) for Gx in self.Gx]
        G_bimodal = self.L1 * tf.math.reduce_sum([tf.math.reduce_sum(tf.multiply(g, 1 - g)) for g in G])
        G_magnitude = self.L2 * tf.math.reduce_sum([tf.math.reduce_sum(g) for g in G])
        return loss + G_bimodal + G_magnitude
        

    def getClassifier(self):
        
        def layer(n):
            
            # Input, with base case
            X = self.X if n == 0 else layer(n - 1)
            X_dropped = tf.nn.dropout(X, 1 - self.dropout)
            
            # Sample from G
            G = clip(self.Gx[n])
            GS = bernoulli(G)
            
            # Activation function
            f = tf.nn.softmax if len(self.layer_sizes) else tf.nn.sigmoid
            return f(self.a * (tf.matmul(X_dropped, tf.multiply(self.W[n], GS)) + self.B[n]))
        
        return layer(len(self.layer_sizes) - 2)
    
    
    def train(self, steps, epochs):
        
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
            sess.run(init)
            
            for step in range(steps):
                
                for epoch in range(epochs):
                    totalLoss = 0.0
                    numBatches = 0
                    
                    for batch_in, batch_out in iter(TrainingBatches(self, self.batch_size)):
                        _, loss = sess.run([self.trainer, self.loss], feed_dict={self.X: batch_in, self.Y:batch_out})
                        totalLoss += loss
                        numBatches += 1
                        
                    loss = totalLoss / numBatches / self.batch_size
                    accuracy = self.validate(sess)
                    
                    print("Step {}, Epoch {}: Loss = {}, Accuracy = {}".format(step, epoch, loss, accuracy))
                        
                        
    def validate(self, sess=None):
        
        if sess == None:
            with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
                output = self.validate(sess)
            return output
        
        pred = tf.nn.softmax(self.classifier)
        percentageCorrect = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(self.Y, 1))))
        accuracy = sess.run([percentageCorrect], feed_dict={self.X: self.validation_in, self.Y: self.validation_out})
        return accuracy

In [13]:
def newNeuralNetwork():
    return NeuralNetwork(**{
        "df": df_training.copy(),
        "df_validation": df_valid.copy(),
        "layer_sizes": (70, 20),
        "output_categories": "rapid",
        "dropout": 0.1,
        "batch_size": 64,
        "initial_sparsity": 0.5})

In [14]:
nn = newNeuralNetwork()

AttributeError: 'int' object has no attribute 'name'

In [ ]:
nn.training_data[0].dtype